<a href="https://colab.research.google.com/github/RodrigoGalvan/MAS_CG_Actividad_Integradora/blob/main/Actividad_Integradora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Actividad 1

Integrantes de equipo:

Rodrigo Galván Paiz | A01721158

Andrés Aguirre Rodríguez | A01284373

Andres Fernando Garza Garcia | A01138704

Daira Adriana Chavarria Rodriguez | A01274745

# Introduccion
En este codigo se presenta el programa que se hizo para el movmiento de los robots para que estos mismos recojan las cajas y las almacenen en el lugar indicado.


In [ ]:
pip install mesa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# 'Model' sirve para definir los atributos a nivel del modelo, maneja los agentes
# 'Agent' es la unidad atómica y puede ser contenido en múltiples instancias en los modelos
from mesa import Agent, Model 

# 'SingleGrid' sirve para forzar a un solo objeto por celda (nuestro objetivo en este "juego")
from mesa.space import MultiGrid
from mesa.space import SingleGrid


# 'SimultaneousActivation' habilita la opción de activar todos los agentes de manera simultanea.
from mesa.time import SimultaneousActivation

# 'DataCollector' permite obtener el grid completo a cada paso (o generación), útil para visualizar
from mesa.datacollection import DataCollector

# 'matplotlib' lo usamos para graficar/visualizar como evoluciona el autómata celular.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2 ** 128

# Definimos los siguientes paquetes para manejar valores númericos: 'numpy' & 'pandas'
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime

# Para usar los datos en Unity
import json

In [ ]:
from pickle import TRUE
class Robot(Agent):
  #Inicializar el robot
  def __init__(self, unique_id, model):
    super().__init__(unique_id, model)
    self.color = 1
    self.count = 0
    self.withBox = False
    self.movingObj = False #Si esta cargando caja o no
    self.dropOff = False #Si tiene que buscar un estante vacio
    self.aroundBox = False
    self.distanceCanTravel = 0 #Distancia a viajar cuando escoge una direccion random

    #Distancia elegida en base si el width o height es mayor
    if(self.model.grid.height > self.model.grid.width):
      self.distanceCanTravel = self.model.grid.height
    else:
      self.distanceCanTravel = self.model.grid.width

    #Una distancia random (Cantidad de movimientos)
    self.randomDistance = self.random.randint(1,self.distanceCanTravel)
    #Direccion random hacia donde ir
    self.randomDirection = self.random.randint(1,4)

  def step(self):
    
    #Buscar por sus vecinos
    neighbours = self.model.grid.get_neighbors(
            self.pos,
            moore=True,
            include_center=True)

    for neighbor in neighbours:
      #Si vecino es una caja y aun no tiene una caja  
      if isinstance(neighbor, Box) and self.movingObj == False:
        self.model.grid.remove_agent(neighbor) #Quitar caja del grid
        self.withBox = True
        self.movingObj = True #Avizar que esta cargando una caja

  def advance(self):
    i = 0
    for cell in self.model.grid.coord_iter():
      cell_content, x, y = cell
      if isinstance(cell_content, Box):
        i += 1

    if not i == 0:
      self.count += 1
      #Movimiento del robot
      self.Move()
    elif (i == 0 and self.withBox) or self.pos[1] == 1 or self.pos[0] == 2:
      self.count += 1
      self.Move()

#Mover el robot
  def Move(self):  
    #Si tiene una caja
    if self.movingObj == True:
      self.color = 9
      #Dependiendo si todavia no llega hasta abajo o si ya tiene que buscar donde dejar la caja
      if self.dropOff == True:
        self.DropOff() #Busca por donde dejar la caja
      else:
        self.MoveDown() #Sigue moviendose hacia abajo
    else:
        #Si no tiene una caja y esta en uno de las lineas donde los robots con cajas recorren entonces se quita
        if self.model.grid.is_cell_empty((self.pos[0] + 1, self.pos[1])) and self.pos[1] != 1 and self.pos[0] == 2:
          self.model.grid.move_agent(self, (self.pos[0]  + 1, self.pos[1]))
        elif self.model.grid.is_cell_empty((self.pos[0], self.pos[1] + 1)) and self.pos[0] == 2:
          self.model.grid.move_agent(self, (self.pos[0], self.pos[1] + 1))
        else:
          #Si esta fuera de los renglones donde los robots con cajas recorren y no tiene caja se mueve
          self.LevyFlight()

#Mover el robot en una direccion
  def LevyFlight(self):
    newPos = self.pos
    #Si aun no recorre todos sus pasos
    if self.randomDistance > 0:
      #Si la celda a donde moverse esta vacia se mueve dependiendo de la direccion escogida random
      if self.randomDirection == 1 and self.model.grid.is_cell_empty((self.pos[0], self.pos[1] + 1)):
        newPos = (self.pos[0], self.pos[1] + 1)
      elif self.randomDirection == 2 and self.model.grid.is_cell_empty((self.pos[0], self.pos[1] - 1)):
        newPos = (self.pos[0], self.pos[1] - 1)
      elif self.randomDirection == 3 and self.model.grid.is_cell_empty((self.pos[0] + 1, self.pos[1])):
        newPos = (self.pos[0] + 1, self.pos[1])
      elif self.randomDirection == 4 and self.model.grid.is_cell_empty((self.pos[0] - 1, self.pos[1])):
        newPos = (self.pos[0] - 1, self.pos[1])
      else:
        #Si no se pudo mover entonces escoge otra direccion
        self.randomDistance = self.random.randint(1,self.distanceCanTravel)
        self.randomDirection = self.random.randint(1,4)

      #Si la posicion escogida no se impone en los renglones donde los robots con caja recorren entonces se mueve
      if(newPos[0] != 2 and newPos[1] != 1):
        self.model.grid.move_agent(self, newPos)
      else:
        #Si no se pudo mover entonces cambia de direccion
        self.randomDistance = self.random.randint(1,self.distanceCanTravel)
        self.randomDirection = self.random.randint(1,4)

      #Se movio y se quita uno de la cantidad de pasos a recorrer
      self.randomDistance -= 1
      
    #Si la distancia es 0 entonces cambiar direccion y cantidad de pasos a recorrer
    if(self.randomDistance <= 0):
      self.randomDistance = self.random.randint(1,self.distanceCanTravel)
      self.randomDirection = self.random.randint(1,4)

#Dejar caja en un estante
  def DropOff(self):
    neighbours = self.model.grid.get_neighbors(
            self.pos,
            moore=False,
            include_center=True)

    for neighbor in neighbours:
      #Busca por vecinos y si es un estante y no tiene 5 cajas entonces entra al if
      if isinstance(neighbor, Pile) and neighbor.color > 4:
        #Quita un color al estante y regresa todo al default
        neighbor.color -= 1
        self.withBox = False
        self.color = 1
        self.movingObj = False
        self.dropOff = False
    
    #Si todavia tiene la caja entonces se mueve uno para la derecha para buscar otro estante
    if self.movingObj == True and self.model.grid.is_cell_empty((self.pos[0], self.pos[1] + 1)):
      self.model.grid.move_agent(self, (self.pos[0], self.pos[1] + 1))

#Mover el robot hacia posicion inicial para dejar la caja en el estante
  def MoveDown(self):
    newPos = self.pos
    moving = False

    move = self.model.grid.get_neighborhood(
                self.pos,
                moore=False,
                include_center=False)
    
    #Si esta intentando rodear caja entonces se va para abajo
    if self.aroundBox == True and self.model.grid.is_cell_empty((self.pos[0], self.pos[1] - 1)):
      newPos = (self.pos[0], self.pos[1] - 1)
      moving = True
      self.aroundBox = False
    else:
      self.aroundBox = False
                
    #Robot se mueve hasta la izquierda si puede
    if self.model.grid.is_cell_empty((self.pos[0], self.pos[1] - 1)):
      newPos = (self.pos[0], self.pos[1] - 1)
      moving = True
    #Si hay obstaculo a la derecha entonces intenta moverse hacia abajo
    elif moving == False and self.pos[0] - 1 != 2:        
      if self.model.grid.is_cell_empty((self.pos[0] - 1, self.pos[1])):
        moving = True
        newPos = (self.pos[0] - 1, self.pos[1])

    #Si Llego hasta la izquierda entonces va hasta abajo
    if self.pos[1] == 1 and self.model.grid.is_cell_empty((self.pos[0] - 1, self.pos[1])) and moving == False:
      newPos = (self.pos[0] - 1, self.pos[1])
      moving = True
    
    #Si no ha llegado hasta la izquierda y no se ha movido entonces intenta ir hacia arriba ya que no pudo hacia abajo
    if self.pos[1] > 1 and moving == False:
       if self.model.grid.is_cell_empty((self.pos[0] + 1, self.pos[1])):
          newPos = (self.pos[0] + 1, self.pos[1])
       else:
          newPos = (self.pos[0], self.pos[1] + 1)
          self.aroundBox = True

    #Si se puede mover se mueve a la ultima posicion propuesta
    if self.model.grid.is_cell_empty(newPos):    
          self.model.grid.move_agent(self, newPos)

    #Si no ha llegado hasta abajo entonces todavia no puede moverse para dejar la caja
    if self.pos[0] == 2 and self.pos[1] == 1:
      self.dropOff = True
    



class Box(Agent):
  def __init__(self, unique_id, model):
    super().__init__(unique_id, model)
    self.color = 2

class Pile(Agent):
  def __init__(self, unique_id, model):
    super().__init__(unique_id, model)
    self.color = 9

class Wall(Agent):
  def __init__(self, unique_id, model):
    super().__init__(unique_id, model)
    self.color = 3





class OrganizerRobotModel(Model):
  def __init__(self, width, height):
      self.grid = SingleGrid(width, height, True)
      self.schedule = SimultaneousActivation(self)
      self.width = width
      self.height = height
      #Cantidad de cajas
      k = int(input("K: "))
      #Cantidad de robots
      nRobots = 5
      #Total de robots en grid
      totalRobots = 0
      #Total de cajas en grid
      totalBoxes = 0
      
      #Poner las paredes y los estantes
      for (content, x, y) in self.grid.coord_iter():
          if((x,y) == (0,y) or (x,y) == (x,0) or (x,y) == (x,self.height-1) or (x,y) == (self.width-1,y)):
            w = Wall((x,y), self)
            self.grid.place_agent(w, (x,y))
            self.schedule.add(w)
          if x == 1 and self.grid.is_cell_empty((x,y)):
            p = Pile((x,y), self)
            self.grid.place_agent(p, (x,y))
            self.schedule.add(p)

      #Poner las cajas en lugares random
      while totalBoxes < k:
        x = self.random.randrange(self.grid.width)
        y = self.random.randrange(self.grid.height)
        if(self.grid.is_cell_empty((x,y))) and (x > 2) and y != 1:
          b = Box((x,y), self)
          self.grid.place_agent(b, (x,y))
          self.schedule.add(b)
          totalBoxes += 1

      #Poner los robots en un lugar random
      while totalRobots < nRobots:
        x = self.random.randrange(self.grid.width)
        y = self.random.randrange(self.grid.height)
        if(self.grid.is_cell_empty((x,y))):
          r = Robot((x,y), self)
          self.grid.place_agent(r, (x,y))
          self.schedule.add(r)
          totalRobots += 1
        
      self.datacollector = DataCollector(
          model_reporters={"Grid": self.get_grid}
      )

  def step(self):
        self.datacollector.collect(self)
        self.schedule.step()
      
  def get_grid(self):
      # Generamos la grid para contener los valores
      grid = np.zeros((self.grid.width, self.grid.height))

      # Asignamos una celda a cada uno de los elementos de la grilla
      for cell in self.grid.coord_iter():
          cell_content, x, y = cell
          #Se itera por cada celda del grid, se sacan los objetos y se pinta la celda
          if isinstance(cell_content, Robot):
            grid[x][y] = cell_content.color
          elif isinstance(cell_content, Box):
            grid[x][y] = cell_content.color
          elif isinstance(cell_content, Wall):
            grid[x][y] = cell_content.color
          elif isinstance(cell_content, Pile):
            grid[x][y] = cell_content.color
          else:
            grid[x][y] = 0
      return grid

  def read_agents(model):
      #Por todas las celdas del grid
      agents_list = []
      for cell in model.grid.coord_iter():
          agent, x, y = cell

          # Si es un robot
          if isinstance(agent, Robot):
              agent_dict = {
                  'id': agent.unique_id,
                  'kind': 'robot',
                  'positionX': agent.pos[0],
                  'positionY': 0,
                  'positionZ': agent.pos[1],
                  'color': agent.colour
              }

          # Si es una caja
          elif isinstance(agent, Box):
              agent_dict = {
                  'id': agent.unique_id,
                  'kind': 'box',
                  'positionX': agent.pos[0],
                  'positionY': 0,
                  'positionZ': agent.pos[1],
                  'color': agent.colour
              }

          # Si es una pila de cajas
          elif isinstance(agent, Pile):
              agent_dict = {
                  'id': agent.unique_id,
                  'kind': 'pile',
                  'positionX': agent.pos[0],
                  'positionY': 0,
                  'positionZ': agent.pos[1],
                  'color': agent.colour
              }
          
          # Si es una pared
          elif isinstance(agent, Wall):
              agent_dict = {
                  'id': agent.unique_id,
                  'kind': 'wall',
                  'positionX': agent.pos[0],
                  'positionY': 0,
                  'positionZ': agent.pos[1],
                  'color': agent.colour
              }

          agents_list.append(agent_dict)
          
          # Convertir a json
          jsondata = json.dumps(agents_list)

      return agents_list
  

In [ ]:
# Definimos el tamaño del Grid
GRID_SIZE = 10

# Definimos el número de generaciones a correr
NUM_GENERATIONS = 300

start_time = time.time()
done = False
model = OrganizerRobotModel(int(input("N: ")) + 2, int(input("M: ")) + 2)
for i in range(NUM_GENERATIONS):
    i = 0
    if(done == False):
      for cell in model.grid.coord_iter():
        cell_content, x, y = cell
        if isinstance(cell_content, Box):
          i += 1
      if(i == 0):
        done = True
        print('Tiempo para recoger cajas:', str(datetime.timedelta(seconds=(time.time() - start_time))))
    model.step()
    # json son utilizados en cada paso 
    # model.read_agents()

print('Tiempo de ejecución:', str(datetime.timedelta(seconds=(time.time() - start_time))))

for cell in model.grid.coord_iter():
  cell_content, x, y = cell
  if isinstance(cell_content, Robot):
      i += 1
      #Se imprime los movimientos de cada  aspiradora
      print('Movimientos Robot {0}: {1}'.format(i, cell_content.count))



N: 15
M: 15
K: 15
Tiempo para recoger cajas: 0:00:05.174688
Tiempo de ejecución: 0:00:05.302289
Movimientos Robot 1: 151
Movimientos Robot 2: 155
Movimientos Robot 3: 133
Movimientos Robot 4: 133
Movimientos Robot 5: 133


In [ ]:
all_grid = model.datacollector.get_model_vars_dataframe()

In [ ]:
%%capture

fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])

#Los colores utilizados
cmap = matplotlib.cm.get_cmap('magma', 10)
cmap = cmap(np.linspace(0, 1, 10))
cmap[0] = np.array([255/256, 255/256, 255/256, 1])  #Piso
cmap[1] = np.array([0/256, 0/256, 0/256, 1])    #Robot
cmap[2] = np.array([150/256, 75/256, 0/256, 1])   #Caja
cmap[3] = np.array([0/256, 0/256, 139/256, 1])   #Pared
cmap[4] = np.array([128/256, 43/256, 26/256, 1])   #Caja 5
cmap[5] = np.array([159/256, 54/256, 32/256, 1])   #Caja 4
cmap[6] = np.array([191/256, 65/256, 38/256, 1])   #Caja 3
cmap[7] = np.array([223/256, 76/256, 45/256, 1])   #Caja 2
cmap[8] = np.array([255/256, 87/256, 51/256, 1])   #Caja 1
cmap[9] = np.array([127/256, 112/256, 83/256, 1])   #Robot con caja



new_cmap = matplotlib.colors.ListedColormap(cmap)

patch = plt.imshow(all_grid.iloc[0][0], cmap=new_cmap)



def animate(i):
    patch.set_data(all_grid.iloc[i][0])
    
anim = animation.FuncAnimation(fig, animate, frames=NUM_GENERATIONS)

In [ ]:
anim